In [16]:
import pandas as pd
import pycountry
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_table
from dash.dependencies import Input, Output
import warnings
import json
import geopandas as gpd
warnings.filterwarnings('ignore')

#### IMPORT DATASETS

In [17]:
# READ IN GLOBAL CO2 EMISSIONS
df_all_co2 = pd.read_csv(r'C:\Users\sydne\Downloads\MyWorld_Co2_Emissions.csv')

df_all_co2.set_index('Year')
df_all_co2['Entity'].unique()

# Get dataset of emissions from most recent year
df_co2 = df_all_co2[df_all_co2['Year'] == 2022]

# Convert Country Names to Str
df_cos = df_co2[~df_co2['Entity'].str.contains('GCP', na=False)]        # Drop columns that have GCP
df_co2.set_index('Year')


,Entity,Code,Annual CO₂ emissions
Year,,,
2022,Afghanistan,AFG,1.055780e+07
2022,Africa,NaN,1.432503e+09
2022,Africa (GCP),NaN,1.432492e+09
2022,Albania,ALB,5.173371e+06
2022,Algeria,DZA,1.845581e+08
...,...,...,...
2022,Wallis and Futuna,WLF,3.001400e+04
2022,World,OWID_WRL,3.729383e+10
2022,Yemen,YEM,1.029614e+07


In [18]:
# READ IN ND GAIN
df_risk = pd.read_csv(r'C:\Users\sydne\Downloads\gain.csv')
drop_yrs = df_risk.columns[2:29].to_list()
for col in drop_yrs:
    df_risk.drop(col, axis = 1, inplace = True)

In [19]:
# Merge the dataframes where the country codes are the same
# this creates new rows for the two rows that match
merged = pd.merge(df_co2, df_risk, left_on='Code', right_on='ISO3', how='outer')

# Eliminate irrelevant entity rows
options = r'(GCP|income|International|excl|(2|part)|Union|World)'
merged = merged[~merged['Entity'].str.contains(options, na=False)]


# drop other miscellaneous rows
dropping_rows = r'(Antarctica|Bermuda|Christmas Island|Faroe Islands|French Polynesia|Greenland|Hong Kong|Kosovo|Macao|New Caledonia|Niue|Palestine|Saint Helena|Saint Pierre and Miquelon|South Sudan|Wallis and Futuna)'
merged = merged[~merged['Entity'].str.contains(dropping_rows, na=False)]
merged = merged[~merged['Entity'].isna() == True]    # dropping nan
merged.drop(index = [1, 3, 7, 21, 25, 28], inplace = True)     # dropping continent rows


# Edit columns
merged.drop(['Year', 'ISO3', 'Name'], axis = 1, inplace = True)
merged.columns = ['Entity', 'Code', 'CO2 Emissions', 'Risk']         

# Get rows from merged df with at least 1 NaN
nan_rows = merged.isna().any(axis = 1)
nan_df = merged[nan_rows]
nan_df = nan_df[~nan_df['Entity'].str.contains(dropping_rows, na=False)]


### Setting Missing Climate Scores

In [20]:
# Setting Liechtenstein's climate score = Austria's 
merged.loc[142].iloc[3] = merged[merged['Entity'] == 'Austria'].iloc[0,3]

# Setting Anguilla's, Satin kitt's, Montserrat's, BVI's climate score = antigua
merged.loc[35].iloc[3] = 49.075398 #merged[merged['Entity'] == 'Antigua and Barbuda'].iloc[0,3]        
merged.loc[193].iloc[3] = merged[merged['Entity'] == 'Antigua and Barbuda'].iloc[0,3]
merged.loc[160].iloc[3] = merged[merged['Entity'] == 'Antigua and Barbuda'].iloc[0,3]
merged.loc[59].iloc[3] = merged[merged['Entity'] == 'Antigua and Barbuda'].iloc[0,3]

# Setting Aruba, Bonaire, Curacao climate score = Barbados
merged.loc[40].iloc[3] = merged[merged['Entity'] == 'Barbados'].iloc[0,3]
merged.loc[55].iloc[3] = merged[merged['Entity'] == 'Barbados'].iloc[0,3]
merged.loc[81].iloc[3]= merged[merged['Entity'] == 'Barbados'].iloc[0,3]

# Setting Turks climate score = Bahamas
merged.loc[230].iloc[3] = merged[merged['Entity'] == 'Bahamas'].iloc[0,3]

# Setting Andorra's climate score = spain
merged.loc[33].iloc[3]= merged[merged['Entity'] == 'Spain'].iloc[0,3]


In [21]:
# resetting index
df = merged.reset_index(drop=True)

### Sync Dataframe with GeoJson

In [22]:
# File path to your GeoJSON
geo_path = r'C:\Users\sydne\Downloads\countries.geo.json'

# Read and modify the GeoJSON
with open(geo_path, 'r') as f:
    geojson_data = json.load(f)  # Load GeoJSON into a Python dictionary

In [23]:
# Reformat country names
new_names = list()
for feature in geojson_data['features']:
    for code in df['Code']:
        if feature['id'] == code:
            # get index of current code
            loc_index = df[df['Code'] == code].index[0]
            value_index = df.index.get_loc(loc_index)
            # replace name
            df.iloc[value_index, 0] = feature['properties']['name']
            new_names.append(feature['properties']['name'])

In [24]:
# Determine which countries present in df but not in geojson
not_present = list()
code_list = df['Code']
found = None
for code in code_list:
    found = False
    for feature in geojson_data['features']:
        if feature['id'] == code:
            found = True
    if found == False:
        not_present.append(code)

print('Number places not found in geojson:', len(not_present))
print('~2/3 of these places are countries; rest are territories')

Number places not found in geojson: 33
~2/3 of these places are countries; rest are territories


In [25]:
#### 33 countries are present in df, but not in geojson
# RETURN TO THIS AFTER INTIAL MAPS CREATED
'''
# Load the full GeoJSON dataset (downloaded from Natural Earth or similar sources)
world = gpd.read_file("path_to_natural_earth_geojson_file.geojson")

# List of ISO3 codes for the 33 places
iso3_codes = [
    'AND', 'AIA', 'ATG', 'ABW', 'BHR', 'BRB', 'BES', 'VGB', 'CPV', 'COM',
    'COK', 'CUW', 'DMA', 'GRD', 'KIR', 'LIE', 'MDV', 'MHL', 'MUS', 'FSM',
    'MSR', 'NRU', 'PLW', 'KNA', 'LCA', 'VCT', 'WSM', 'STP', 'SYC', 'SGP',
    'TON', 'TCA', 'TUV'
]

# Filter the dataset for the specified ISO3 codes
filtered = world[world['ISO_A3'].isin(iso3_codes)]

# Save the filtered data to a new GeoJSON file
filtered.to_file("filtered_locations.geojson", driver="GeoJSON")

print("Filtered GeoJSON file created successfully!")
'''

'\n# Load the full GeoJSON dataset (downloaded from Natural Earth or similar sources)\nworld = gpd.read_file("path_to_natural_earth_geojson_file.geojson")\n\n# List of ISO3 codes for the 33 places\niso3_codes = [\n    \'AND\', \'AIA\', \'ATG\', \'ABW\', \'BHR\', \'BRB\', \'BES\', \'VGB\', \'CPV\', \'COM\',\n    \'COK\', \'CUW\', \'DMA\', \'GRD\', \'KIR\', \'LIE\', \'MDV\', \'MHL\', \'MUS\', \'FSM\',\n    \'MSR\', \'NRU\', \'PLW\', \'KNA\', \'LCA\', \'VCT\', \'WSM\', \'STP\', \'SYC\', \'SGP\',\n    \'TON\', \'TCA\', \'TUV\'\n]\n\n# Filter the dataset for the specified ISO3 codes\nfiltered = world[world[\'ISO_A3\'].isin(iso3_codes)]\n\n# Save the filtered data to a new GeoJSON file\nfiltered.to_file("filtered_locations.geojson", driver="GeoJSON")\n\nprint("Filtered GeoJSON file created successfully!")\n'

In [26]:
#### Some countries lacking emission data (because included in geojson, but not df)
#### Some countries have emission data, but lacking location (because included in df but not geojson)


# Adding counties only in geojson to df         
add_countries = {'Entity': ['Antarctica', 'French Southern and Antarctic Lands', 'Bermuda', 'Falkland Islands', 'Greenland', 'French Guiana', 'Serbia and Montenegro', 'New Caledonia', 'Puerto Rico', 'Western Sahara', 'South Sudan', 'Palestine'], 
                'Code': ['ATA', 'ATF', 'BMU', 'FLK', 'GRL', 'GUF', 'CS-KM', 'NCL', 'PRI', 'ESH', 'SSD', 'PSE'], 
                'CO2 Emissions': [np.nan, np.nan,np.nan, np.nan,np.nan, np.nan,np.nan, np.nan,np.nan, np.nan,np.nan, np.nan], 
                'Risk': [np.nan, np.nan, np.nan, np.nan,np.nan, np.nan,np.nan, np.nan,np.nan, np.nan,np.nan, np.nan]}
df = pd.concat([df, pd.DataFrame(add_countries)])    

# Remove irrelevant entries from geojson (-99)
geojson_data['features'] = [feature for feature in geojson_data['features'] if feature['id'] != '-99']

In [27]:
# Merge the data with GeoJSON features
for feature in geojson_data['features']:
    geo_code = feature['id']  # assuming 'id' contains country code in GeoJSON
    if geo_code in df['Code'].values:
        # Merge the data by adding both emission and risk to the properties of the GeoJSON feature
        emission = df.loc[df['Code'] == geo_code, 'CO2 Emissions'].values[0]
        risk = df.loc[df['Code'] == geo_code, 'Risk'].values[0]
        feature['properties']['CO2 Emissions'] = emission
        feature['properties']['Risk'] = risk

In [28]:
# THIS CELL CAN BE DELETED IF GEOJSON DATA IS FOUND FOR THE REMAINING 33 TERRITORIES/COUNTRIES IN THE DF
# Extract the relevant data from geojson_data
data = []
for feature in geojson_data['features']:
    data.append({
        'id': feature['id'],  # Country code or ID
        'name': feature['properties']['name'],  # Country name
        'CO2 Emissions': feature['properties']['CO2 Emissions'],  # CO2 Emissions data
        'Risk': feature['properties']['Risk']  # Risk data for hover
    })

geo_df = pd.DataFrame(data)
geo_df.columns = ['Code', 'Entity', 'CO2 Emissions', 'Risk']

In [29]:
# Cloropleth for CO2 

fig1 = px.choropleth(geo_df,
    geojson=geojson_data,
    locations=[feature['id'] for feature in geojson_data['features']],  # List of country codes
    color=[feature['properties']['CO2 Emissions'] for feature in geojson_data['features']],  # List of emissions values
    color_continuous_scale='Viridis',  # Color scale for the emissions
    labels={'color': 'CO2 Emissions'},  # Label for the color scale
    hover_data = ['CO2 Emissions', 'Risk'],
    hover_name = 'Entity',
    title = 'Global CO2 Emissions'
)

In [30]:
# Choropleth for Risk
fig2 = px.choropleth(geo_df, 
    geojson = geojson_data,
    locations = [feature['id'] for feature in geojson_data['features']],  # List of country codes
    color=[feature['properties']['Risk'] for feature in geojson_data['features']],  # List of emissions values
    color_continuous_scale='Viridis',  # Color scale for the emissions
    labels={'color': 'Risk'},  # Label for the color scale
    hover_data = ['Risk', 'CO2 Emissions'],
    hover_name = 'Entity',
    title = 'Global Climate Risk'
)

### Dash Application 

In [31]:
# create a Dash Application
app = JupyterDash(__name__)

# Create the layout of the application (all graphs, dropdowns, text, charts)
app.layout = html.Div(children = [

    # Title of dashboard
    html.H1('Global Climate Risk Analysis', style = {'textAlign':'center', 'fontSize':40, 'color':'black'}),
    # Choropleth Maps
    dcc.Graph(
        id = 'emissions_choropleth', 
        figure = fig1,  
    ),
    dcc.Graph(
        id = 'risk_choropleth', 
        figure = fig2,
    )
])

# Create callback to update graph when input changes
@app.callback(
    # Output and input do not take keywords   [first element = id, second = component_property
    [Output('emissions_choropleth', 'figure'), Output('risk_choropleth', 'figure')]
)

# function needed after the callback function
def show_choropleths():
    return fig1, fig2

app.run_server('external')

Dash app running on http://127.0.0.1:8050/
